In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import kurtosis, skew
import plotly.graph_objects as go

# -------- Data Loading -------- #
etf_close_price = pd.read_csv("../../Downloads/etf_close_data.csv")
etf_covariance_matrix = pd.read_csv("../../Downloads/etf_covariance_matrix.csv", index_col=0)
etf_returns = etf_close_price.set_index("Date").pct_change().dropna()

# -------- CVaR Objective Function -------- #
def cvar_objective(weights, returns, alpha=0.95):
    portfolio_returns = returns @ weights
    var = np.percentile(portfolio_returns, 100 * (1 - alpha))
    cvar = -np.mean(portfolio_returns[portfolio_returns <= var])
    return cvar

# -------- Portfolio Optimizer -------- #
def optimize_portfolio_cvar(returns_df, alpha=0.95):
    n_assets = returns_df.shape[1]
    init_weights = np.ones(n_assets) / n_assets
    bounds = [(0, 1) for _ in range(n_assets)]  # long-only
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    result = minimize(
        fun=cvar_objective,
        x0=init_weights,
        args=(returns_df, alpha),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result.x

# -------- Metric Calculator -------- #
def compute_metrics(portfolio_returns, alpha=0.95):
    daily_mean = np.mean(portfolio_returns)
    annual_return = daily_mean * 252
    geometric_return = (np.prod(1 + portfolio_returns))**(252 / len(portfolio_returns)) - 1
    min_daily_return = np.min(portfolio_returns)
    daily_vol = np.std(portfolio_returns)
    annual_sharpe = daily_mean / daily_vol * np.sqrt(252)
    skewness = skew(portfolio_returns)
    excess_kurtosis = kurtosis(portfolio_returns)
    cumulative = (1 + portfolio_returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    max_dd = drawdown.min()
    rolling_10 = cumulative.rolling(10).max()
    drawdown_10d = (cumulative - rolling_10) / rolling_10
    max_dd_10 = drawdown_10d.min()
    var_95 = -np.percentile(portfolio_returns, 5)
    cvar_95 = -np.mean(portfolio_returns[portfolio_returns <= -var_95])
    return pd.DataFrame({
        "Mean Daily Return": [daily_mean],
        "Annualized Return": [annual_return],
        "Geometric Return": [geometric_return],
        "Minimum Daily Return": [min_daily_return],
        "Volatility (daily)": [daily_vol],
        "Sharpe Ratio (annual)": [annual_sharpe],
        "Skewness": [skewness],
        "Kurtosis (excess)": [excess_kurtosis],
        "Max Drawdown": [max_dd],
        "Max 10-Day Drawdown": [max_dd_10],
        "VaR 95% (1-day)": [var_95],
        "CVaR 95% (1-day)": [cvar_95],
    })

# -------- Main Execution Flow -------- #
def run_cvar_portfolio_analysis(returns_df, alpha=0.95):
    weights = optimize_portfolio_cvar(returns_df, alpha)
    portfolio_returns = returns_df @ weights
    metrics = compute_metrics(portfolio_returns, alpha)
    return weights, portfolio_returns, metrics

# Run base case analysis
weights, portfolio_returns, metrics = run_cvar_portfolio_analysis(etf_returns)

# -------- Visualization with Plotly -------- #
cumulative_returns = (1 + portfolio_returns).cumprod()
fig_cum = go.Figure()
fig_cum.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines',
                             name='CVaR Portfolio', line=dict(color='royalblue')))
fig_cum.update_layout(title='Cumulative Return of CVaR Portfolio',
                      xaxis_title='Date', yaxis_title='Cumulative Return', template='plotly_white')
fig_cum.show()

rolling_max = cumulative_returns.rolling(window=10, min_periods=1).max()
drawdown_10d = (cumulative_returns - rolling_max) / rolling_max
fig_dd = go.Figure()
fig_dd.add_trace(go.Scatter(x=drawdown_10d.index, y=drawdown_10d, mode='lines',
                            name='10-Day Rolling Drawdown', line=dict(color='firebrick')))
fig_dd.update_layout(title='10-Day Rolling Maximum Drawdown',
                     xaxis_title='Date', yaxis_title='Drawdown', template='plotly_white')
fig_dd.show()

# -------- Sensitivity Analysis -------- #
alphas = [0.90, 0.925, 0.95, 0.975, 0.99]
sensitivity_results = []
for alpha in alphas:
    _, _, metrics = run_cvar_portfolio_analysis(etf_returns, alpha)
    metrics["CVaR Alpha Level"] = alpha
    sensitivity_results.append(metrics)

sensitivity_df = pd.concat(sensitivity_results, ignore_index=True)

# Print as a table
print("\n=== CVaR Sensitivity Analysis ===")
print(sensitivity_df.to_string(index=False))

# Optional: Save to CSV
# sensitivity_df.to_csv("cvar_sensitivity_results.csv", index=False)



=== CVaR Sensitivity Analysis ===
 Mean Daily Return  Annualized Return  Geometric Return  Minimum Daily Return  Volatility (daily)  Sharpe Ratio (annual)  Skewness  Kurtosis (excess)  Max Drawdown  Max 10-Day Drawdown  VaR 95% (1-day)  CVaR 95% (1-day)  CVaR Alpha Level
          0.000129           0.032522          0.032430             -0.039434            0.002189               0.935916 -0.953251          47.294667     -0.079500            -0.064431         0.002744          0.004629             0.900
          0.000129           0.032432          0.032339             -0.039267            0.002186               0.934402 -0.964973          45.772905     -0.077340            -0.063405         0.002753          0.004621             0.925
          0.000131           0.032935          0.032857             -0.038827            0.002188               0.948257 -0.914638          44.170525     -0.075503            -0.062763         0.002758          0.004619             0.950
          0.0

In [3]:
# -------- Packages -------- #
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import kurtosis, skew
import plotly.graph_objects as go

# -------- Data Loading -------- #
etf_close_price = pd.read_csv("../../Downloads/etf_close_data.csv")
etf_covariance_matrix = pd.read_csv("../../Downloads/etf_covariance_matrix.csv", index_col=0)
etf_returns = etf_close_price.set_index("Date").pct_change().dropna()

# -------- Metric Calculator -------- #
def compute_metrics(portfolio_returns, alpha=0.95):
    daily_mean = np.mean(portfolio_returns)
    annual_return = daily_mean * 252
    geometric_return = (np.prod(1 + portfolio_returns))**(252 / len(portfolio_returns)) - 1
    min_daily_return = np.min(portfolio_returns)
    daily_vol = np.std(portfolio_returns)
    annual_sharpe = daily_mean / daily_vol * np.sqrt(252)
    skewness = skew(portfolio_returns)
    excess_kurtosis = kurtosis(portfolio_returns)
    cumulative = (1 + portfolio_returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    max_dd = drawdown.min()
    rolling_10 = cumulative.rolling(10).max()
    drawdown_10d = (cumulative - rolling_10) / rolling_10
    max_dd_10 = drawdown_10d.min()
    var_95 = -np.percentile(portfolio_returns, 5)
    cvar_95 = -np.mean(portfolio_returns[portfolio_returns <= -var_95])
    return pd.DataFrame({
        "Mean Daily Return": [daily_mean],
        "Annualized Return": [annual_return],
        "Geometric Return": [geometric_return],
        "Minimum Daily Return": [min_daily_return],
        "Volatility (daily)": [daily_vol],
        "Sharpe Ratio (annual)": [annual_sharpe],
        "Skewness": [skewness],
        "Kurtosis (excess)": [excess_kurtosis],
        "Max Drawdown": [max_dd],
        "Max 10-Day Drawdown": [max_dd_10],
        "VaR 95% (1-day)": [var_95],
        "CVaR 95% (1-day)": [cvar_95],
    })

# -------- CVaR + Return Tradeoff Objective Function -------- #
def return_cvar_tradeoff_objective(weights, returns, alpha=0.95, lambda_=0.5):
    portfolio_returns = returns @ weights
    expected_return = np.mean(portfolio_returns[-30:])  # average of last 30 days
    var = np.percentile(portfolio_returns, 100 * (1 - alpha))
    cvar = -np.mean(portfolio_returns[portfolio_returns <= var])
    objective = -(expected_return - lambda_ * cvar)  # negative because minimize()
    return objective

# -------- Portfolio Optimizer -------- #
def optimize_portfolio_return_cvar(returns_df, alpha=0.95, lambda_=0.5):
    n_assets = returns_df.shape[1]
    init_weights = np.ones(n_assets) / n_assets
    bounds = [(0, 1) for _ in range(n_assets)]  # long-only
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    result = minimize(
        fun=return_cvar_tradeoff_objective,
        x0=init_weights,
        args=(returns_df, alpha, lambda_),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result.x

# -------- Main Execution Flow -------- #
def run_return_cvar_tradeoff_analysis(returns_df, alpha=0.95, lambda_=0.5):
    weights = optimize_portfolio_return_cvar(returns_df, alpha, lambda_)
    portfolio_returns = returns_df @ weights
    metrics = compute_metrics(portfolio_returns, alpha)
    return weights, portfolio_returns, metrics

# -------- Run the Analysis -------- #
# parameters
alpha = 0.95
lambda_ = 0.5

# optimization
tradeoff_weights, tradeoff_portfolio_returns, tradeoff_metrics = run_return_cvar_tradeoff_analysis(etf_returns, alpha, lambda_)

# -------- Visualization -------- #
# Cumulative returns plot
tradeoff_cumulative_returns = (1 + tradeoff_portfolio_returns).cumprod()
fig_tradeoff_cum = go.Figure()
fig_tradeoff_cum.add_trace(go.Scatter(x=tradeoff_cumulative_returns.index, y=tradeoff_cumulative_returns, mode='lines',
                                      name='Return-CVaR Tradeoff Portfolio', line=dict(color='green')))
fig_tradeoff_cum.update_layout(title='Cumulative Return: Return-CVaR Tradeoff Portfolio',
                               xaxis_title='Date', yaxis_title='Cumulative Return', template='plotly_white')
fig_tradeoff_cum.show()

# Drawdown plot
rolling_max_tradeoff = tradeoff_cumulative_returns.rolling(window=10, min_periods=1).max()
drawdown_10d_tradeoff = (tradeoff_cumulative_returns - rolling_max_tradeoff) / rolling_max_tradeoff
fig_dd_tradeoff = go.Figure()
fig_dd_tradeoff.add_trace(go.Scatter(x=drawdown_10d_tradeoff.index, y=drawdown_10d_tradeoff, mode='lines',
                                     name='10-Day Rolling Drawdown', line=dict(color='darkred')))
fig_dd_tradeoff.update_layout(title='10-Day Rolling Maximum Drawdown: Tradeoff Portfolio',
                              xaxis_title='Date', yaxis_title='Drawdown', template='plotly_white')
fig_dd_tradeoff.show()

# -------- Metrics Output -------- #
print("\n=== Return-CVaR Tradeoff Portfolio Metrics ===")
print(tradeoff_metrics.to_string(index=False))



=== Return-CVaR Tradeoff Portfolio Metrics ===
 Mean Daily Return  Annualized Return  Geometric Return  Minimum Daily Return  Volatility (daily)  Sharpe Ratio (annual)  Skewness  Kurtosis (excess)  Max Drawdown  Max 10-Day Drawdown  VaR 95% (1-day)  CVaR 95% (1-day)
          0.000142           0.035874          0.035858             -0.042021            0.002254               1.002484 -1.245886          51.937034     -0.078107            -0.067055         0.002855          0.004742


In [ ]:
import ace_tools as tools


ModuleNotFoundError: No module named 'ace_tools'

In [4]:
# -------- Packages -------- #
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import kurtosis, skew
import plotly.graph_objects as go

# -------- Data Loading -------- #
etf_close_price = pd.read_csv("../../Downloads/etf_close_data.csv")
etf_covariance_matrix = pd.read_csv("../../Downloads/etf_covariance_matrix.csv", index_col=0)
etf_returns = etf_close_price.set_index("Date").pct_change().dropna()

# -------- Metric Calculator -------- #
def compute_metrics(portfolio_returns, alpha=0.95):
    daily_mean = np.mean(portfolio_returns)
    annual_return = daily_mean * 252
    geometric_return = (np.prod(1 + portfolio_returns))**(252 / len(portfolio_returns)) - 1
    min_daily_return = np.min(portfolio_returns)
    daily_vol = np.std(portfolio_returns)
    annual_sharpe = daily_mean / daily_vol * np.sqrt(252)
    skewness = skew(portfolio_returns)
    excess_kurtosis = kurtosis(portfolio_returns)
    cumulative = (1 + portfolio_returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    max_dd = drawdown.min()
    rolling_10 = cumulative.rolling(10).max()
    drawdown_10d = (cumulative - rolling_10) / rolling_10
    max_dd_10 = drawdown_10d.min()
    var_95 = -np.percentile(portfolio_returns, 5)
    cvar_95 = -np.mean(portfolio_returns[portfolio_returns <= -var_95])
    return pd.DataFrame({
        "Mean Daily Return": [daily_mean],
        "Annualized Return": [annual_return],
        "Geometric Return": [geometric_return],
        "Minimum Daily Return": [min_daily_return],
        "Volatility (daily)": [daily_vol],
        "Sharpe Ratio (annual)": [annual_sharpe],
        "Skewness": [skewness],
        "Kurtosis (excess)": [excess_kurtosis],
        "Max Drawdown": [max_dd],
        "Max 10-Day Drawdown": [max_dd_10],
        "VaR 95% (1-day)": [var_95],
        "CVaR 95% (1-day)": [cvar_95],
    })

# -------- CVaR Minimization Objective Function -------- #
def cvar_objective(weights, returns, alpha=0.95):
    portfolio_returns = returns @ weights
    var = np.percentile(portfolio_returns, 100 * (1 - alpha))
    cvar = -np.mean(portfolio_returns[portfolio_returns <= var])
    return cvar

# -------- Return-CVaR Tradeoff Objective Function -------- #
def return_cvar_tradeoff_objective(weights, returns, alpha=0.95, lambda_=0.5):
    portfolio_returns = returns @ weights
    expected_return = np.mean(portfolio_returns[-30:])  # rolling 30-day mean
    var = np.percentile(portfolio_returns, 100 * (1 - alpha))
    cvar = -np.mean(portfolio_returns[portfolio_returns <= var])
    objective = -(expected_return - lambda_ * cvar)  # maximize expected - λ * CVaR
    return objective

# -------- Portfolio Optimizers -------- #
def optimize_min_cvar(returns_df, alpha=0.95):
    n_assets = returns_df.shape[1]
    init_weights = np.ones(n_assets) / n_assets
    bounds = [(0, 1) for _ in range(n_assets)]
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    result = minimize(
        fun=cvar_objective,
        x0=init_weights,
        args=(returns_df, alpha),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result.x

def optimize_return_cvar_tradeoff(returns_df, alpha=0.95, lambda_=0.5):
    n_assets = returns_df.shape[1]
    init_weights = np.ones(n_assets) / n_assets
    bounds = [(0, 1) for _ in range(n_assets)]
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    result = minimize(
        fun=return_cvar_tradeoff_objective,
        x0=init_weights,
        args=(returns_df, alpha, lambda_),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result.x

# -------- Main Execution -------- #
alpha = 0.95
lambda_ = 0.5

# Optimize both portfolios
min_cvar_weights = optimize_min_cvar(etf_returns, alpha)
tradeoff_weights = optimize_return_cvar_tradeoff(etf_returns, alpha, lambda_)

# Compute returns
min_cvar_portfolio_returns = etf_returns @ min_cvar_weights
tradeoff_portfolio_returns = etf_returns @ tradeoff_weights

# Metrics
min_cvar_metrics = compute_metrics(min_cvar_portfolio_returns, alpha)
tradeoff_metrics = compute_metrics(tradeoff_portfolio_returns, alpha)

# -------- Visualization: Cumulative Returns -------- #
min_cvar_cum_returns = (1 + min_cvar_portfolio_returns).cumprod()
tradeoff_cum_returns = (1 + tradeoff_portfolio_returns).cumprod()

fig_cum = go.Figure()
fig_cum.add_trace(go.Scatter(x=min_cvar_cum_returns.index, y=min_cvar_cum_returns, mode='lines',
                             name='Min CVaR Portfolio', line=dict(color='royalblue')))
fig_cum.add_trace(go.Scatter(x=tradeoff_cum_returns.index, y=tradeoff_cum_returns, mode='lines',
                             name='Return-CVaR Tradeoff Portfolio', line=dict(color='green')))
fig_cum.update_layout(title='Cumulative Return Comparison',
                      xaxis_title='Date', yaxis_title='Cumulative Return',
                      template='plotly_white')
fig_cum.show()

# -------- Visualization: 10-Day Rolling Drawdowns -------- #
rolling_max_min_cvar = min_cvar_cum_returns.rolling(window=10, min_periods=1).max()
drawdown_10d_min_cvar = (min_cvar_cum_returns - rolling_max_min_cvar) / rolling_max_min_cvar

rolling_max_tradeoff = tradeoff_cum_returns.rolling(window=10, min_periods=1).max()
drawdown_10d_tradeoff = (tradeoff_cum_returns - rolling_max_tradeoff) / rolling_max_tradeoff

fig_dd = go.Figure()
fig_dd.add_trace(go.Scatter(x=drawdown_10d_min_cvar.index, y=drawdown_10d_min_cvar, mode='lines',
                            name='Min CVaR Portfolio', line=dict(color='royalblue')))
fig_dd.add_trace(go.Scatter(x=drawdown_10d_tradeoff.index, y=drawdown_10d_tradeoff, mode='lines',
                            name='Return-CVaR Tradeoff Portfolio', line=dict(color='green')))
fig_dd.update_layout(title='10-Day Rolling Maximum Drawdown Comparison',
                     xaxis_title='Date', yaxis_title='Drawdown',
                     template='plotly_white')
fig_dd.show()

# -------- Metrics Output -------- #
print("\n=== Min CVaR Portfolio Metrics ===")
print(min_cvar_metrics.to_string(index=False))
print("\n=== Return-CVaR Tradeoff Portfolio Metrics ===")
print(tradeoff_metrics.to_string(index=False))



=== Min CVaR Portfolio Metrics ===
 Mean Daily Return  Annualized Return  Geometric Return  Minimum Daily Return  Volatility (daily)  Sharpe Ratio (annual)  Skewness  Kurtosis (excess)  Max Drawdown  Max 10-Day Drawdown  VaR 95% (1-day)  CVaR 95% (1-day)
          0.000131           0.032935          0.032857             -0.038827            0.002188               0.948257 -0.914638          44.170525     -0.075503            -0.062763         0.002758          0.004619

=== Return-CVaR Tradeoff Portfolio Metrics ===
 Mean Daily Return  Annualized Return  Geometric Return  Minimum Daily Return  Volatility (daily)  Sharpe Ratio (annual)  Skewness  Kurtosis (excess)  Max Drawdown  Max 10-Day Drawdown  VaR 95% (1-day)  CVaR 95% (1-day)
          0.000142           0.035874          0.035858             -0.042021            0.002254               1.002484 -1.245886          51.937034     -0.078107            -0.067055         0.002855          0.004742


In [5]:
# -------- Packages -------- #
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import kurtosis, skew
import plotly.graph_objects as go

# -------- Data Loading -------- #
etf_close_price = pd.read_csv("../../Downloads/etf_close_data.csv")
etf_returns = etf_close_price.set_index("Date").pct_change().dropna()

# -------- Metric Calculator -------- #
def compute_metrics(portfolio_returns):
    daily_mean = np.mean(portfolio_returns)
    daily_vol = np.std(portfolio_returns)
    annualized_return = daily_mean * 252
    sharpe_ratio = (daily_mean / daily_vol) * np.sqrt(252)
    return annualized_return, sharpe_ratio

# -------- Return-CVaR Tradeoff Objective Function -------- #
def return_cvar_tradeoff_objective(weights, returns, alpha, lambda_, momentum_window):
    portfolio_returns = returns @ weights
    expected_return = np.mean(portfolio_returns[-momentum_window:])  # rolling momentum
    var = np.percentile(portfolio_returns, 100 * (1 - alpha))
    cvar = -np.mean(portfolio_returns[portfolio_returns <= var])
    objective = -(expected_return - lambda_ * cvar)  # maximize expected - λ * CVaR
    return objective

# -------- Portfolio Optimizer -------- #
def optimize_portfolio(returns_df, alpha=0.95, lambda_=0.5, momentum_window=30):
    n_assets = returns_df.shape[1]
    init_weights = np.ones(n_assets) / n_assets
    bounds = [(0, 1) for _ in range(n_assets)]
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    result = minimize(
        fun=return_cvar_tradeoff_objective,
        x0=init_weights,
        args=(returns_df, alpha, lambda_, momentum_window),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    return result.x

# -------- Sensitivity Analysis -------- #
momentum_windows = [5, 10, 15, 30]
lambdas = [0.2, 0.5, 0.8, 1.3]
alpha = 0.95

sensitivity_results = []
cumulative_returns_dict = {}

for momentum_window in momentum_windows:
    for lambda_ in lambdas:
        weights = optimize_portfolio(etf_returns, alpha=alpha, lambda_=lambda_, momentum_window=momentum_window)
        portfolio_returns = etf_returns @ weights
        cumulative_returns = (1 + portfolio_returns).cumprod()
        cumulative_returns_dict[f"momentum_{momentum_window}_lambda_{lambda_}"] = cumulative_returns

        ann_return, sharpe = compute_metrics(portfolio_returns)
        sensitivity_results.append({
            "Momentum Window": momentum_window,
            "Lambda": lambda_,
            "Annualized Return": ann_return,
            "Sharpe Ratio": sharpe
        })

sensitivity_df = pd.DataFrame(sensitivity_results)

# -------- Plot Cumulative Returns -------- #
fig_cum = go.Figure()
for label, cum_returns in cumulative_returns_dict.items():
    fig_cum.add_trace(go.Scatter(x=cum_returns.index, y=cum_returns, mode='lines', name=label))
fig_cum.update_layout(title='Cumulative Returns for Different Momentum & Lambda Combinations',
                      xaxis_title='Date', yaxis_title='Cumulative Return',
                      template='plotly_white', legend_title='(Momentum, Lambda)')
fig_cum.show()

# -------- Scatter Plot of Metrics -------- #
fig_metrics = go.Figure()
for idx, row in sensitivity_df.iterrows():
    label = f"m={row['Momentum Window']}, λ={row['Lambda']}"
    fig_metrics.add_trace(go.Scatter(
        x=[row['Sharpe Ratio']], y=[row['Annualized Return']],
        mode='markers+text',
        text=[label], textposition="top center",
        marker=dict(size=10)
    ))
fig_metrics.update_layout(title='Sharpe Ratio vs Annualized Return',
                           xaxis_title='Sharpe Ratio',
                           yaxis_title='Annualized Return',
                           template='plotly_white')
fig_metrics.show()

# -------- Print Sensitivity Table -------- #
print("\n=== Sensitivity Analysis Results ===")
print(sensitivity_df.to_string(index=False))


Output hidden; open in https://colab.research.google.com to view.